In [3]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

In [4]:
# Mesma separacao do exercicio anterior (SVM) salvas em csv
X_train = pd.read_csv('../data/X_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_train = pd.read_csv('../data/y_train.csv')
y_test = pd.read_csv('../data/y_test.csv')

KeyboardInterrupt: 

In [ ]:
# normalizando os dados
#juntando datasets para normalizar em conjunto
X = pd.concat([X_train,X_test])

# Normalizando 0 e 1
types = X['tipo']
X = X.drop(['tipo'],axis=1)
X = (X - X.min()) / (X.max() - X.min())

y = pd.concat([y_train, y_test])

In [40]:
def train_pred_classifier(X_train, X_test, y_train, y_test,
                          hl_size, activation, alpha,epocas):
    
    kf = KFold(n_splits=10)
    # prepara classificador
    clf = MLPClassifier(solver='lbfgs', # resolve com hasseane, mais rapido
                        alpha=alpha,
                        hidden_layer_sizes=hl_size,
                        activation = activation,
                        max_iter = epocas,
                        random_state=1)
    # treina
    clf.fit(X_train,y_train)
    
    # predict
    y_pred = clf.predict(X_test)
    
    # calcula erros
    p = precision_score(y_test, y_pred, average='micro')
    r = recall_score(y_test, y_pred, average='micro')
    auc = roc_auc_score(y_test, y_pred, average='micro')
    return p,r,auc
    

In [41]:
tipo = 'Todas as Caracteristicas'
camadas_ocultas = 
  (10,1), #  uma camada oculta com 10 neuronios
  (10, 15), # duas camadas ocultas com 10 e 15 neuronios
  (10, 10, 10) # tres camadas ocultas 10, 30, 10
]
funcao_ativacao = ['logistic', 'relu']
alpha = [0.001, 0.01, 0.1, 0.2, 0.3]
epocas = [500, 1000, 2000, 5000]


SyntaxError: invalid syntax (<ipython-input-41-8e79e89b3fa8>, line 2)

In [42]:
all_dicts = []
for t in tipo:
    for layer_conf in camadas_ocultas:
        for f in funcao_ativacao:
            for a in alpha:
                for epoch in epocas:
                    p,r,auc = train_pred_classifier(X_train, X_test,
                                          y_train, y_test,
                                          layer_conf, f, a,epoch)
                    one_line_response = {
                        'tipo' : tipo,
                        'camadas_ocultas' : layer_conf,
                        'funcao_ativacao' : f,
                        'num_epocas' : epoch,
                        'alpha': a,
                        'precision': p,
                        'recall': r,
                        'auc': auc
                    }
                    all_dicts.append(one_line_response)
                    break

0.001
(10, 1)


/home/favoretti/.local/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.01
(10, 1)
0.1
(10, 1)
0.2
(10, 1)


KeyboardInterrupt: 

In [43]:
all_dicts

[{'alpha': 0.001,
  'auc': 0.5,
  'camadas_ocultas': (10, 1),
  'funcao_ativacao': 'logistic',
  'num_epocas': 500,
  'precision': 0.47727272727272729,
  'recall': 0.47727272727272729,
  'tipo': 'Todas as Caracteristicas'},
 {'alpha': 0.01,
  'auc': 0.5,
  'camadas_ocultas': (10, 1),
  'funcao_ativacao': 'logistic',
  'num_epocas': 500,
  'precision': 0.47727272727272729,
  'recall': 0.47727272727272729,
  'tipo': 'Todas as Caracteristicas'},
 {'alpha': 0.1,
  'auc': 0.5,
  'camadas_ocultas': (10, 1),
  'funcao_ativacao': 'logistic',
  'num_epocas': 500,
  'precision': 0.47727272727272729,
  'recall': 0.47727272727272729,
  'tipo': 'Todas as Caracteristicas'}]

In [44]:
pd.DataFrame(all_dicts).to_csv('final_results.csv', index=False)